# Preprocessing - School Related

Handling school-related data first addresses the non-unique properties caused by multiple nearby schools
_________________

## Landing Layer

In [85]:
import pandas as pd

In [86]:
# Load the complete dataset
rental_df_landing = pd.read_csv('../data/landing/rental_df_landing.csv')

rental_df_landing = rental_df_landing.rename(columns={'distance/米': 'distance/m'})

In [87]:
SCHOOL_RELATED = ['id', 'rent', 'address', 'educationLevel', 'name', 'distance/m', 'year', 'gender', 'get_type']

# Filter the DataFrame to only include school-related features
rental_df_schoolRelated_landing = rental_df_landing[SCHOOL_RELATED]

rental_df_schoolRelated_landing.to_csv('../data/landing/rental_df_schoolRelated_landing.csv', index=False)

#print(rental_df_schoolRelated_landing.head())

## Raw Layer

In [88]:
import re
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

- range

In [89]:
# Check the initial range of 'distance/m'
distance_range_initial = rental_df_schoolRelated_landing['distance/m'].agg(['min', 'max'])
print(f"Initial range of distances: {distance_range_initial['min']:.2f} to {distance_range_initial['max']:.2f} meters")

# Extract unique values for certain features (year, gender, get_type)
features_to_check = ['year', 'gender', 'get_type']
unique_feature_values = {}

for feature in features_to_check:
    unique_feature_values[feature] = rental_df_schoolRelated_landing[feature].unique().tolist()

# Print unique values for each feature
for feature, values in unique_feature_values.items():
    print(f"Unique values for '{feature}': {values}")

Initial range of distances: 16.42 to 96330.26 meters
Unique values for 'year': ['Prep-6', nan, 'Prep-9', '10-12', '7-12', '3-9', '9-12', 'Prep-12', 'U', 'Unknown', '11-12', '3-12', '7-9', '1-12', '7-11', '5-12', '7', '10-11', '7-10', 'Prep-7', 'U, Prep-6', 'U, Prep-12', 'Prep-8', '11', '3-10', '7-8', '6-10', 'Prep-11', '1-10', 'K-12', 'U, 7-12', '8-12', 'Prep-5', '6-12', 'Prep-3', 'K-6', '3-6', 'U, Prep-9', 'Prep-10', '5']
Unique values for 'gender': ['CoEd', nan, 'Girls', 'Boys']
Unique values for 'get_type': ['Government', 'Catholic', 'Private']


In [90]:
min_distance_allowed = 10  # Minimum allowed distance in meters
max_distance_allowed = 10000  # Maximum allowed distance in meters

# Create a copy of the DataFrame for range checking
rental_df_schoolRelated_valid_distance = rental_df_schoolRelated_landing.copy()

# Replace distances outside the allowed range with NaN
rental_df_schoolRelated_valid_distance.loc[
    (rental_df_schoolRelated_valid_distance['distance/m'] < min_distance_allowed) | 
    (rental_df_schoolRelated_valid_distance['distance/m'] > max_distance_allowed), 
    'distance/m'] = np.nan

# Check the range of distances after range filtering
distance_range_filtered = rental_df_schoolRelated_valid_distance['distance/m'].agg(['min', 'max'])
print(f"Filtered range of distances: {distance_range_filtered['min']:.2f} to {distance_range_filtered['max']:.2f} meters")

# Function to select the most common year for properties with multiple years listed
def select_most_frequent_year(year_column_value, year_count_dict):
    if isinstance(year_column_value, str) and ',' in year_column_value:  # Check for multiple years
        years_list = [year.strip() for year in year_column_value.split(',')]
        # Find the most common year based on frequency
        most_common_year = max(years_list, key=lambda year: year_count_dict[year])
        return most_common_year
    return year_column_value  # If only one year, return as-is

# Create a list of all years for counting frequency
all_years_list = [year.strip() for row in rental_df_schoolRelated_valid_distance['year'] if isinstance(row, str) for year in row.split(',')]
year_frequency = Counter(all_years_list)

# Apply the function to select the most frequent year for each property
rental_df_schoolRelated_valid_distance['year'] = \
    rental_df_schoolRelated_valid_distance['year'].apply(lambda year_value: select_most_frequent_year(year_value, year_frequency))

print(f"Unique values for 'year' after cleaning: {rental_df_schoolRelated_valid_distance['year'].unique().tolist()}")

Filtered range of distances: 16.42 to 9998.26 meters
Unique values for 'year' after cleaning: ['Prep-6', nan, 'Prep-9', '10-12', '7-12', '3-9', '9-12', 'Prep-12', 'U', 'Unknown', '11-12', '3-12', '7-9', '1-12', '7-11', '5-12', '7', '10-11', '7-10', 'Prep-7', 'Prep-8', '11', '3-10', '7-8', '6-10', 'Prep-11', '1-10', 'K-12', '8-12', 'Prep-5', '6-12', 'Prep-3', 'K-6', '3-6', 'Prep-10', '5']


- duplicates

The duplicates typically appear because the dataset is scraped using property feature filters, where the same property may be listed under multiple filters.

In [91]:
# Group by 'id' and 'name' to check for duplicate entries and count occurrences
duplicate_check_school = rental_df_schoolRelated_valid_distance.groupby(['id', 'name']).size().reset_index(name='count')

duplicate_entries = duplicate_check_school[duplicate_check_school['count'] > 1]

# Display the first few rows of the duplicate entries
duplicate_entries.head()

,id,name,count
0,6861939,Docklands Primary School,2
1,6861939,Eltham College - Lonsdale Street Campus,2
2,6861939,Haileybury College - City Boys,2
3,6861939,Haileybury College - City Girls,2
4,6861939,Hester Hornbrook Academy,2


In [92]:
rental_df_schoolRelated_no_duplicates = rental_df_schoolRelated_valid_distance.drop_duplicates(subset=['id', 'name'])

rental_df_schoolRelated_no_duplicates.shape

(86073, 9)

- representations

In [93]:
# Function to clean the 'rent' value by extracting the numeric amount and removing commas
def extract_rent_amount(value):
    match = re.search(r'\$(\d{1,3}(?:,\d{3})*|\d+)(?:\.\d+)?', value)  # Find rent in the format $X,XXX or $XXX
    
    if match:
        return match.group(1).replace(',', '')  # Remove commas to return a clean number
    else:
        return np.nan  # Return NaN if no match is found

rental_df_schoolRelated_clean_rent = rental_df_schoolRelated_no_duplicates.copy()

# Apply the rent cleaning function to the 'rent' column
rental_df_schoolRelated_clean_rent['rent'] = rental_df_schoolRelated_clean_rent['rent'].apply(extract_rent_amount)

rental_df_schoolRelated_clean_rent['rent'] = pd.to_numeric(rental_df_schoolRelated_clean_rent['rent'], errors='coerce')

# Apply a reasonable range for 'rent' values, marking anything outside 10-10000 as NaN
rental_df_schoolRelated_clean_rent.loc[
    (rental_df_schoolRelated_clean_rent['rent'] < 10) | 
    (rental_df_schoolRelated_clean_rent['rent'] > 10000), 
    'rent'] = np.nan

- outliers

performing the train-test split before handling outliers and missing values ensures that the test set remains untouched by any decisions or patterns learned during the data cleaning process

In [94]:
unique_property_ids = rental_df_schoolRelated_clean_rent['id'].unique()

# Split the unique property IDs into train and test sets
train_ids, test_ids = train_test_split(unique_property_ids, test_size=0.2, random_state=42)

# Create the training and test datasets by filtering the original dataset based on the train and test IDs
train_df_schoolRelated_no_outliers = rental_df_schoolRelated_clean_rent[rental_df_schoolRelated_clean_rent['id'].isin(train_ids)]
test_df_schoolRelated_no_outliers = rental_df_schoolRelated_clean_rent[rental_df_schoolRelated_clean_rent['id'].isin(test_ids)]

In [95]:
# Aggregate rent by property (id), taking the mean rent per property in the training set
rent_agg_train = train_df_schoolRelated_no_outliers.groupby('id').agg({
    'rent': 'mean',  
}).reset_index()

# Calculate the Interquartile Range (IQR) for the aggregated rent values in the training set
Q1 = rent_agg_train['rent'].quantile(0.25)
Q3 = rent_agg_train['rent'].quantile(0.75)
IQR = Q3 - Q1

n_train = rent_agg_train.shape[0]

# Apply the formula: sqrt(log(n)) - 0.5 * IQR if n > 100, otherwise use 1.5 * IQR
if n_train > 100:
    upper_threshold = Q3 + (np.sqrt(np.log(n_train)) - 0.5) * IQR
    lower_threshold = Q1 - (np.sqrt(np.log(n_train)) - 0.5) * IQR
else:
    upper_threshold = Q3 + 1.5 * IQR
    lower_threshold = Q1 - 1.5 * IQR

# Replace outliers in the rent values with NaN based on the calculated thresholds in the training set
rent_agg_train['rent_cleaned'] = rent_agg_train['rent'].where(
    (rent_agg_train['rent'] <= upper_threshold) & (rent_agg_train['rent'] >= lower_threshold), 
    np.nan
)

# Merge the cleaned rent values back into the original training DataFrame
train_df_schoolRelated_no_outliers = train_df_schoolRelated_no_outliers.merge(
    rent_agg_train[['id', 'rent_cleaned']], 
    on='id', 
    how='left'
)

# Replace original 'rent' values with the cleaned ones in the training set
train_df_schoolRelated_no_outliers['rent'] = train_df_schoolRelated_no_outliers['rent_cleaned']

# Apply the same thresholds to the test set
# Aggregate rent by property (id) in the test set
rent_agg_test = test_df_schoolRelated_no_outliers.groupby('id').agg({
    'rent': 'mean',  # Option to use median if necessary
}).reset_index()

# Replace outliers in the rent values with NaN in the test set using the same thresholds as the training set
rent_agg_test['rent_cleaned'] = rent_agg_test['rent'].where(
    (rent_agg_test['rent'] <= upper_threshold) & (rent_agg_test['rent'] >= lower_threshold), 
    np.nan
)

# Merge the cleaned rent values back into the original test DataFrame
test_df_schoolRelated_no_outliers = test_df_schoolRelated_no_outliers.merge(
    rent_agg_test[['id', 'rent_cleaned']], 
    on='id', 
    how='left'
)

# Replace original 'rent' values with the cleaned ones in the test set
test_df_schoolRelated_no_outliers['rent'] = test_df_schoolRelated_no_outliers['rent_cleaned']

train_df_schoolRelated_no_outliers = train_df_schoolRelated_no_outliers.drop(columns=['rent_cleaned'])
test_df_schoolRelated_no_outliers = test_df_schoolRelated_no_outliers.drop(columns=['rent_cleaned'])

- missing values

In [96]:
train_df_schoolRelated_nomiss = train_df_schoolRelated_no_outliers.copy()
test_df_schoolRelated_nomiss = test_df_schoolRelated_no_outliers.copy()

# Print the number of missing values at the start for key columns in both train and test sets
print("Initial missing values in training set:")
print(train_df_schoolRelated_nomiss[['rent', 'distance/m', 'year', 'gender']].isna().sum())

Initial missing values in training set:
rent           3710
distance/m     1400
year          15850
gender        16597
dtype: int64


In [97]:
# Fill missing 'rent' with the average rent from the training set
average_rent_train = train_df_schoolRelated_nomiss['rent'].mean()
train_df_schoolRelated_nomiss['rent'].fillna(average_rent_train, inplace=True)

# Fill missing 'distance/m' with the average distance from the training set
average_distance_train = train_df_schoolRelated_nomiss['distance/m'].mean()
train_df_schoolRelated_nomiss['distance/m'].fillna(average_distance_train, inplace=True)

# For 'year', treat 'Unknown' as NaN, then fill NaN with the mode
train_df_schoolRelated_nomiss['year'].replace('Unknown', np.nan, inplace=True)
year_mode_train = train_df_schoolRelated_nomiss['year'].mode()[0]
train_df_schoolRelated_nomiss['year'].fillna(year_mode_train, inplace=True)

# Fill missing 'gender' with the mode from the training set
gender_mode_train = train_df_schoolRelated_nomiss['gender'].mode()[0]
train_df_schoolRelated_nomiss['gender'].fillna(gender_mode_train, inplace=True)

# Apply the same logic to the test set (test_df_schoolRelated_no_outliers)
test_df_schoolRelated_nomiss['rent'].fillna(average_rent_train, inplace=True)
test_df_schoolRelated_nomiss['distance/m'].fillna(average_distance_train, inplace=True)

test_df_schoolRelated_nomiss['year'].replace('Unknown', np.nan, inplace=True)
test_df_schoolRelated_nomiss['year'].fillna(year_mode_train, inplace=True)

test_df_schoolRelated_nomiss['gender'].fillna(gender_mode_train, inplace=True)

print("\nMissing values after filling in training set:")
print(train_df_schoolRelated_nomiss[['rent', 'distance/m', 'year', 'gender']].isna().sum())

print("\nMissing values after filling in test set:")
print(test_df_schoolRelated_nomiss[['rent', 'distance/m', 'year', 'gender']].isna().sum())


Missing values after filling in training set:
rent          0
distance/m    0
year          0
gender        0
dtype: int64

Missing values after filling in test set:
rent          0
distance/m    0
year          0
gender        0
dtype: int64


In [98]:
# Save the cleaned training DataFrame to the specified path
train_df_schoolRelated_nomiss.to_csv('../data/raw/train_df_schoolRelated_raw.csv', index=False)

# Save the cleaned test DataFrame to the specified path
test_df_schoolRelated_nomiss.to_csv('../data/raw/test_df_schoolRelated_raw.csv', index=False)

# Print confirmation messages
print("Training set saved to '../data/raw/train_df_schoolRelated_raw.csv'")
print("Test set saved to '../data/raw/test_df_schoolRelated_raw.csv'")

Training set saved to '../data/raw/train_df_schoolRelated_raw.csv'
Test set saved to '../data/raw/test_df_schoolRelated_raw.csv'


## Curated Layer

In [99]:
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.preprocessing import MinMaxScaler

- Route Distance Calculation

Typically, we prefer to focus on the route distance to schools rather than the straight-line distance.

In [ ]:
# Function to get route distances from an API (e.g., Google Maps Distance Matrix API)
def get_route_distances(api_key, origin, destinations):
    endpoint = "https://maps.googleapis.com/maps/api/distancematrix/json"
    
    params = {
        'origins': origin, 
        'destinations': '|'.join(destinations),  
        'key': api_key,
        'mode': 'driving'  # Change to 'walking' or 'bicycling' if needed
    }
    
    response = requests.get(endpoint, params=params)
    
    if response.status_code == 200:
        results = response.json()
        return results['rows'][0]['elements']  # Returns distances to each destination
    else:
        return None

# Step 1: Combine the training and test sets
combined_df_schoolRelated = pd.concat([train_df_schoolRelated_nomiss, test_df_schoolRelated_nomiss])

# Step 2: Group by 'id' and 'address', aggregating nearby school names
combined_rental_df_perProperty = combined_df_schoolRelated.groupby(['id', 'address'])['name'].apply(list).reset_index()

to_school_routeDistances = []

# Step 3: Calculate route distances for each property
for index, property in combined_rental_df_perProperty.iterrows():
    property_address = property['address']
    school_names = property['name']

    # Get distances using the Google API
    distances = get_route_distances('___api_key___', property_address, school_names)
    
    if distances:
        for i, school in enumerate(school_names):
            distance = {
                'id': property['id'],
                'address': property_address,
                'name': school,
                'route_distance/m': distances[i]['distance']['value'] if 'distance' in distances[i] else None
            }
            to_school_routeDistances.append(distance)

# Step 4: Convert the distances list to a DataFrame
to_school_routeDistances_df = pd.DataFrame(to_school_routeDistances)

# Step 5: Save the route distances DataFrame to a CSV file
file_path = '../data/distance/to_school_routeDistances.csv'
to_school_routeDistances_df.to_csv(file_path, index=False)

In [28]:
# to_school_routeDistances_df = pd.read_csv('../data/distance/to_school_routeDistances.csv')

In [100]:
train_df_schoolRelated_nomiss['order'] = train_df_schoolRelated_nomiss.index
test_df_schoolRelated_nomiss['order'] = test_df_schoolRelated_nomiss.index

# Merge school route distances into both train and test sets
train_df_schoolRelated_w_routeDistance = pd.merge(train_df_schoolRelated_nomiss, 
                                                  to_school_routeDistances_df[['id', 'address', 'name', 'route_distance/m']], 
                                                  on=['id', 'address', 'name'], 
                                                  how='inner')

test_df_schoolRelated_w_routeDistance = pd.merge(test_df_schoolRelated_nomiss, 
                                                 to_school_routeDistances_df[['id', 'address', 'name', 'route_distance/m']], 
                                                 on=['id', 'address', 'name'], 
                                                 how='inner')

# Restore the original order in both train and test sets
train_df_schoolRelated_w_routeDistance = train_df_schoolRelated_w_routeDistance.sort_values(by='order').reset_index(drop=True)
test_df_schoolRelated_w_routeDistance = test_df_schoolRelated_w_routeDistance.sort_values(by='order').reset_index(drop=True)

train_df_schoolRelated_nomiss = train_df_schoolRelated_nomiss.drop(columns=['order'])
test_df_schoolRelated_nomiss = test_df_schoolRelated_nomiss.drop(columns=['order'])

train_df_schoolRelated_w_routeDistance = train_df_schoolRelated_w_routeDistance.drop(columns=['order'])
test_df_schoolRelated_w_routeDistance = test_df_schoolRelated_w_routeDistance.drop(columns=['order'])


In [101]:
def fill_missing_route_distances(row, property_avg_ratios, dataset_avg_ratio):
    if pd.isna(row['route_distance/m']): 
        property_avg_ratio = property_avg_ratios.get(row['id'], np.nan)
        if not pd.isna(property_avg_ratio): 
            return row['distance/m'] * property_avg_ratio
        else:  
            return row['distance/m'] * dataset_avg_ratio
    return row['route_distance/m']

# Feature engineering on training data
def clean_school_route_distances_train(df):
    df_cleaned = df.copy()
    
    # Create distance_ratio
    df_cleaned['distance_ratio'] = df.apply(
        lambda row: row['route_distance/m'] / row['distance/m'] if not pd.isna(row['route_distance/m']) else np.nan, axis=1
    )
    
    # Apply thresholds for distance_ratio
    upper_ratio_threshold = 5
    lower_ratio_threshold = 0.2

    df_cleaned.loc[
        (df_cleaned['distance_ratio'] > upper_ratio_threshold) |
        (df_cleaned['distance_ratio'] < lower_ratio_threshold), 
        ['route_distance/m', 'distance_ratio']] = np.nan

    # Calculate property and dataset averages based on training data
    property_avg_ratios = df_cleaned.groupby('id')['distance_ratio'].apply(
        lambda x: np.mean([val for val in x if not pd.isna(val)])
    )
    
    dataset_avg_ratio = df_cleaned['distance_ratio'].apply(
        lambda x: x if not pd.isna(x) else np.nan
    ).mean()
    
    # Fill missing route distances
    df_cleaned['route_distance/m'] = df_cleaned.apply(
        fill_missing_route_distances, axis=1, property_avg_ratios=property_avg_ratios, dataset_avg_ratio=dataset_avg_ratio
    )
    
    # Drop the 'distance_ratio' column
    df_cleaned = df_cleaned.drop(columns=['distance_ratio'])
    
    return df_cleaned, property_avg_ratios, dataset_avg_ratio

# Feature engineering on test data using stats from training data
def clean_school_route_distances_test(df, property_avg_ratios, dataset_avg_ratio):
    df_cleaned = df.copy()

    # Create distance_ratio
    df_cleaned['distance_ratio'] = df.apply(
        lambda row: row['route_distance/m'] / row['distance/m'] if not pd.isna(row['route_distance/m']) else np.nan, axis=1
    )

    # Apply thresholds for distance_ratio
    upper_ratio_threshold = 5
    lower_ratio_threshold = 0.2

    df_cleaned.loc[
        (df_cleaned['distance_ratio'] > upper_ratio_threshold) |
        (df_cleaned['distance_ratio'] < lower_ratio_threshold), 
        ['route_distance/m', 'distance_ratio']] = np.nan

    # Fill missing route distances using training stats
    df_cleaned['route_distance/m'] = df_cleaned.apply(
        fill_missing_route_distances, axis=1, property_avg_ratios=property_avg_ratios, dataset_avg_ratio=dataset_avg_ratio
    )

    # Drop the 'distance_ratio' column
    df_cleaned = df_cleaned.drop(columns=['distance_ratio'])

    return df_cleaned

# Clean train dataset and compute ratios
train_df_schoolRelated_w_cleaned_routeDistance, property_avg_ratios_train, dataset_avg_ratio_train = clean_school_route_distances_train(train_df_schoolRelated_w_routeDistance)

# Clean test dataset using the ratios from the train dataset
test_df_schoolRelated_w_cleaned_routeDistance = clean_school_route_distances_test(test_df_schoolRelated_w_routeDistance, property_avg_ratios_train, dataset_avg_ratio_train)

/Users/shanjunqi/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/shanjunqi/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


This process fills missing route distances by calculating property-specific and dataset-wide ratios in the training data and applying these ratios to fill missing values in both the training and test sets, ensuring consistency.

- School Scoring

We predict the school score by using a Random Forest model to estimate the rental prices of its nearby properties based on school attributes.

In [102]:
# Define the categorical features related to schools
school_related_categorical = ['educationLevel', 'year', 'gender', 'get_type']

# Initialize OneHotEncoder
school_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the training dataset
school_encoded_train = school_encoder.fit_transform(train_df_schoolRelated_w_cleaned_routeDistance[school_related_categorical])

# Transform the test dataset using the same encoder (no refit)
school_encoded_test = school_encoder.transform(test_df_schoolRelated_w_cleaned_routeDistance[school_related_categorical])

# Get encoded feature names
encoded_feature_names = school_encoder.get_feature_names_out(school_related_categorical)

# Create DataFrames for encoded train and test sets
train_df_schoolRelated_encoded = pd.DataFrame(school_encoded_train, columns=encoded_feature_names)
train_df_schoolRelated_encoded['id'] = train_df_schoolRelated_w_cleaned_routeDistance['id']
train_df_schoolRelated_encoded['rent'] = train_df_schoolRelated_w_cleaned_routeDistance['rent']
train_df_schoolRelated_encoded['route_distance/m'] = train_df_schoolRelated_w_cleaned_routeDistance['route_distance/m']

test_df_schoolRelated_encoded = pd.DataFrame(school_encoded_test, columns=encoded_feature_names)
test_df_schoolRelated_encoded['id'] = test_df_schoolRelated_w_cleaned_routeDistance['id']
test_df_schoolRelated_encoded['rent'] = test_df_schoolRelated_w_cleaned_routeDistance['rent']
test_df_schoolRelated_encoded['route_distance/m'] = test_df_schoolRelated_w_cleaned_routeDistance['route_distance/m']

# Organize columns
train_df_schoolRelated_encoded = train_df_schoolRelated_encoded[['id', 'rent'] + list(encoded_feature_names) + ['route_distance/m']]
test_df_schoolRelated_encoded = test_df_schoolRelated_encoded[['id', 'rent'] + list(encoded_feature_names) + ['route_distance/m']]

In [103]:
# Cross-validation on training set
school_related_features = list(encoded_feature_names)

# X and y for the training set
X_train = train_df_schoolRelated_encoded[school_related_features]
y_train = train_df_schoolRelated_encoded['rent']

# Define cross-validation and model
cv = KFold(n_splits=5, shuffle=True, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validated predictions on the training set (for score calculation)
predicted_rent_train = cross_val_predict(rf_model, X_train, y_train, cv=cv)

# Fit the model on the full training set and predict the test set
rf_model.fit(X_train, y_train)

# X for the test set
X_test = test_df_schoolRelated_encoded[school_related_features]

# Predict rent for the test set
predicted_rent_test = rf_model.predict(X_test)

# Assign the cross-validated predictions back to the training set as 'school_score'
train_df_schoolRelated_encoded['school_score'] = predicted_rent_train

# Assign the predictions for the test set as 'school_score'
test_df_schoolRelated_encoded['school_score'] = predicted_rent_test

# Save the updated DataFrames with the school score
train_df_schoolRelated_w_schoolScore = train_df_schoolRelated_encoded.copy()
test_df_schoolRelated_w_schoolScore = test_df_schoolRelated_encoded.copy()

- Calculating Education Index

In [104]:
# Initialize a MinMaxScaler for normalizing the school scores
scaler = MinMaxScaler()

# Normalize school scores for the training set
school_score_reshaped_train = train_df_schoolRelated_w_schoolScore['school_score'].values.reshape(-1, 1)
school_score_normalized_train = scaler.fit_transform(school_score_reshaped_train)
train_df_schoolRelated_w_schoolScore['school_score'] = school_score_normalized_train

# Normalize school scores for the test set using the scaler fitted on the training data
school_score_reshaped_test = test_df_schoolRelated_w_schoolScore['school_score'].values.reshape(-1, 1)
school_score_normalized_test = scaler.transform(school_score_reshaped_test)
test_df_schoolRelated_w_schoolScore['school_score'] = school_score_normalized_test

# Normalize reverse distance for both train and test sets using MinMaxScaler
distance_scaler = MinMaxScaler()

# Normalize reverse distance for the training set
train_df_schoolRelated_w_schoolScore['reverse_distance/m'] = 1 - distance_scaler.fit_transform(
    train_df_schoolRelated_w_schoolScore[['route_distance/m']]
)

# Normalize reverse distance for the test set using the scaler fitted on training data
test_df_schoolRelated_w_schoolScore['reverse_distance/m'] = 1 - distance_scaler.transform(
    test_df_schoolRelated_w_schoolScore[['route_distance/m']]
)

In [105]:
# Calculate education index equally important score for both train and test sets
train_df_schoolRelated_w_schoolScore['education_index_equally_important_score'] = (
    0.5 * train_df_schoolRelated_w_schoolScore['school_score'] + 0.5 * train_df_schoolRelated_w_schoolScore['reverse_distance/m']
)

test_df_schoolRelated_w_schoolScore['education_index_equally_important_score'] = (
    0.5 * test_df_schoolRelated_w_schoolScore['school_score'] + 0.5 * test_df_schoolRelated_w_schoolScore['reverse_distance/m']
)

# Normalize the education index equally important score for both train and test sets
scaler = MinMaxScaler()

# Normalize for the training set
train_df_schoolRelated_w_schoolScore['education_index_equally_important_score'] = scaler.fit_transform(
    train_df_schoolRelated_w_schoolScore[['education_index_equally_important_score']]
)

# Normalize for the test set using the scaler fitted on the training set
test_df_schoolRelated_w_schoolScore['education_index_equally_important_score'] = scaler.transform(
    test_df_schoolRelated_w_schoolScore[['education_index_equally_important_score']]
)

In [106]:
# Function to compute the education index by summing the scores of the top 5 nearest schools
def compute_education_index(group):
    nearest_schools = group.sort_values(by='education_index_equally_important_score', ascending=False).head(5)
    education_index = np.sum(nearest_schools['education_index_equally_important_score'])
    return education_index

# Apply this function to compute the education index for each property in train and test sets
train_education_index = train_df_schoolRelated_w_schoolScore.groupby('id').apply(lambda group: compute_education_index(group))
test_education_index = test_df_schoolRelated_w_schoolScore.groupby('id').apply(lambda group: compute_education_index(group))

In [81]:
# Merge the education index into the train and test rental datasets
train_df_schoolRelated_w_educationIndex = train_df_schoolRelated_nomiss.copy()
test_df_schoolRelated_w_educationIndex = test_df_schoolRelated_nomiss.copy()

school_related_columns_to_drop = ['educationLevel', 'name', 'distance/m', 'year', 'gender', 'get_type']

train_df_schoolRelated_w_educationIndex = train_df_schoolRelated_w_educationIndex.drop(columns=school_related_columns_to_drop)
train_df_schoolRelated_w_educationIndex = train_df_schoolRelated_w_educationIndex.drop_duplicates(subset='id', keep='first')

test_df_schoolRelated_w_educationIndex = test_df_schoolRelated_w_educationIndex.drop(columns=school_related_columns_to_drop)
test_df_schoolRelated_w_educationIndex = test_df_schoolRelated_w_educationIndex.drop_duplicates(subset='id', keep='first')

# Merge the education index back into the train and test rental datasets
train_df_schoolRelated_w_educationIndex = pd.merge(
    train_df_schoolRelated_w_educationIndex, 
    train_education_index.rename('educationIndex'), 
    left_on='id', right_index=True, how='inner'
)

test_df_schoolRelated_w_educationIndex = pd.merge(
    test_df_schoolRelated_w_educationIndex, 
    test_education_index.rename('educationIndex'), 
    left_on='id', right_index=True, how='inner'
)

In [82]:
train_df_schoolRelated_w_educationIndex.to_csv('../data/curated/train_df_schoolRelated_curated.csv', index=False)

test_df_schoolRelated_w_educationIndex.to_csv('../data/curated/test_df_schoolRelated_curated.csv', index=False)

print("Training set saved to '../data/curated/train_df_schoolRelated_curated.csv'")
print("Test set saved to '../data/curated/test_df_schoolRelated_curated.csv'")

Training set saved to '../data/curated/train_df_schoolRelated_curated.csv'
Test set saved to '../data/curated/test_df_schoolRelated_curated.csv'
